## Table of Contents 🚀

1. Introduction
2. Installation
3. Basic Usage
   - Prompt Generation
   - Chat Interface
4. Advanced Features
   - Middleware Integration
   - Streaming Responses


## 1. Introduction 📖
Declarai turns your Python code into LLM tasks, allowing you to easily integrate LLM into your existing codebase. It operates on a simple principle: just define a Python function/class. By annotating this function with docstrings and type hints, you provide a clear instruction set for the AI model without any additional effort.

Once you've declared your function, Declarai intelligently compiles the function's docstrings and type hints into a prompt for the AI model, ensuring the model understands exactly what's required.

After executing the task, Declarai retrieves the AI's response and parses it, translating it back into the declared return type of your Python function. This eliminates any manual parsing or post-processing on your part.

Declarai Keeps It Native: At its core, Declarai is about embracing native Python practices. You don't need to learn a new syntax or adapt to a different coding paradigm. Just write Python functions as you always have, and let Declarai handle the AI integration seamlessly.



## 2. Installation

In [ ]:
!pip install declarai

In [22]:
%set_env DECLARAI_OPENAI_API_KEY=<your-key>

env: DECLARAI_OPENAI_API_KEY=<your-key>


##3. Basic Usage

In [23]:
import declarai

In [24]:
gpt_35 = declarai.openai(model='gpt-3.5-turbo')

@gpt_35.task
def rank_by_severity(message: str) -> int:
    """
    Rank the severity of the provided message by it's urgency.
    Urgency is ranked on a scale of 1-5, with 5 being the most urgent.
    :param message: The message to rank
    :return: The urgency of the message
    """

In [25]:
rank_by_severity(message="The server is down!")

5

In [ ]:
rank_by_severity(message="How was your weekend?")

1

In [ ]:
@gpt_35.experimental.chat
class SQLChat:
    """
    You are a sql assistant. You are helping a user to write a sql query.
    You should first know what sql syntax the user wants to use. It can be mysql, postgresql, sqllite, etc.
    If the user says something that is completely not related to SQL, you should say "I don't understand. I'm here to help you write a SQL query."
    After you provide the user with a query, you should ask the user if they need anything else.
    """

    greeting = "Hey dear SQL User. Hope you are doing well today. I am here to help you write a SQL query. Let's get started!. What SQL syntax would you like to use? It can be mysql, postgresql, sqllite, etc."


chat = SQLChat()

In [ ]:
print(chat.send("What is right join?"))

A right join is a type of join operation in SQL that returns all the rows from the right table and the matching rows from the left table. If there is no match, NULL values are returned for the columns of the left table. It is also known as a right outer join.

Here's an example of a right join:

```sql
SELECT *
FROM table1
RIGHT JOIN table2
ON table1.column = table2.column;
```

This query will return all the rows from table2 and the matching rows from table1 based on the specified join condition. If there is no match, NULL values will be returned for the columns of table1.

Is there anything else I can assist you with?


In [ ]:
print(chat.send("How is it different from left join"))

A right join and a left join are similar in that they both return all the rows from one table and the matching rows from another table. However, the key difference lies in which table's rows are returned in their entirety.

In a left join, all the rows from the left table are returned, along with the matching rows from the right table. If there is no match, NULL values are returned for the columns of the right table.

In a right join, all the rows from the right table are returned, along with the matching rows from the left table. If there is no match, NULL values are returned for the columns of the left table.

Here's an example to illustrate the difference:

Consider two tables, `customers` and `orders`, with a common column `customer_id`. 

A left join would return all the rows from the `customers` table and the matching rows from the `orders` table:

```sql
SELECT *
FROM customers
LEFT JOIN orders
ON customers.customer_id = orders.customer_id;
```

A right join, on the other hand, 

##4a. Middleware

In [ ]:
from declarai.middleware import LoggingMiddleware
from typing import List

@gpt_35.task(middlewares=[LoggingMiddleware])
def multi_value_extraction(text: str) -> List[str]:
    """
    Extract the phone numbers from the provided text
    :param text: content to extract phone number from
    :return: The phone numbers that where identified in the input text
    """

You can see there is a dict printed stating different information about the task run

In [ ]:
multi_value_extraction(
    text="Hey jenny,\nyou can call me at 124-3435-132.\n"
         "you can also reach me at +43-938-243-223"
)

{'task_name': 'multi_value_extraction', 'llm_model': 'gpt-3.5-turbo-0613', 'template': '{\'messages\': [system: You are a REST api endpoint.You only answer in JSON structures\nwith a single key named \'declarai_result\', nothing else.\nThe expected format is:\n"declarai_result": List[string]  # The phone numbers that where identified in the input text, user: Extract the phone numbers from the provided text\nInputs:\ntext: {text}\n\n]}', 'call_kwargs': "{'text': 'Hey jenny,\\nyou can call me at 124-3435-132.\\nyou can also reach me at +43-938-243-223'}", 'compiled_template': '{\'messages\': [system: You are a REST api endpoint.You only answer in JSON structures\nwith a single key named \'declarai_result\', nothing else.\nThe expected format is:\n"declarai_result": List[string]  # The phone numbers that where identified in the input text, user: Extract the phone numbers from the provided text\nInputs:\ntext: Hey jenny,\nyou can call me at 124-3435-132.\nyou can also reach me at +43-938-2

['124-3435-132', '+43-938-243-223']

4b. Streaming

In [ ]:
@gpt_35.task(streaming=True)
def summarize_text(text: str) -> str:
  """
  Summarize the following long text
  """

res = summarize_text(text="""
    The English Wikipedia is the primary[a] English-language edition of Wikipedia, an online encyclopedia. It was created by Jimmy Wales and Larry Sanger on January 15, 2001, as Wikipedia's first edition.

English Wikipedia is hosted alongside other language editions by the Wikimedia Foundation, an American nonprofit organization. Its content is written independently of other editions[1] in various varieties of English, aiming to stay consistent within articles. Its internal newspaper is The Signpost.

English Wikipedia is the most-read version of Wikipedia[2] and has the most articles of any edition, at 6,708,664 as of September 2023.[3] It contains 10.9% of articles in all Wikipedias,[3] although it lacks millions of articles found in other editions.[1] The edition's one-billionth edit was made on January 13, 2021.[4]

English Wikipedia, often as a stand-in for Wikipedia overall, has been praised for its enablement of the democratization of knowledge, extent of coverage, unique structure, culture, and reduced degree of commercial bias. It has been criticized for exhibiting systemic bias, particularly gender bias against women and ideological bias.[5][6] While its reliability was frequently criticized in the 2000s, it has improved over time, receiving greater praise in the late 2010s and early 2020s,[7][5][8][b] having become an important fact-checking site.[9][10] English Wikipedia has been characterized as having less cultural bias than other language editions due to its broader editor base.[2]

Articles

Screenshot of Wikipedia's article on Earth, January 18, 2023
Opening English Wikipedia's main page with Mozilla Firefox 99 on Ubuntu 20.04
Editors of the English Wikipedia have pioneered some ideas as conventions, policies or features which were later adopted by Wikipedia editions in some of the other languages. These ideas include "featured articles",[11] the neutral-point-of-view policy,[12] navigation templates,[13] the sorting of short "stub" articles into sub-categories,[14] dispute resolution mechanisms such as mediation and arbitration,[15] and weekly collaborations.[16]

It surpassed six million articles on 23 January 2020.[17] In November 2022, the total volume of the compressed texts of its articles amounted to 20 gigabytes.[18]

The edition's one-billionth edit was made on 13 January 2021 by Ser Amantio di Nicolao (Steven Pruitt) who as of that date is the user with the highest number of edits on the English Wikipedia, at over four million.[4] Currently, there are 6,708,664 articles created with 905,208 files. The encyclopedia is home to 10.9% of articles in all Wikipedias (down from more than 50% in 2003).[19][20] The English Wikipedia currently has 46,091,912 registered accounts of which 888 are administrators.

"""
)

In [ ]:
for chunk in res:
  print(chunk.response)